Test of alchemical calculater class

In [1]:
target_mol = "C 0 0 0; O 0 0 1.1"
dft_functional = "pbe0"  # "lda,vwn"
name_basis_set = "def2-TZVP"
nuc_charges_vec = [1, -1]

In [2]:
from aqa.alch_calc import alchemical_calculator as ac

In [3]:
ac_mol = ac(target_mol, name_basis_set, dft_functional, sites=[0,1], guess="1e",
            flag_response_property=True)

converged SCF energy = -113.229636035167


In [4]:
ac_mol.calc_all_derivatives()

In [5]:
import inspect
print(inspect.getfile(ac_mol.mol.with_rinv_at_nucleus))

/home/takafumi/program/pyscf/pyscf/gto/mole.py


In [6]:
ac_mol.get_elec_APDFT0(nuc_charges_vec)

-136.32100523894925

In [7]:
ac_mol.get_elec_APDFT1(nuc_charges_vec)

-129.72319608741495

In [8]:
ac_mol.get_elec_APDFT2(nuc_charges_vec)

-132.91969683831977

In [9]:
ac_mol.get_elec_APDFT3(nuc_charges_vec)

-132.9326120392592

In [10]:
ac_mol.get_relative_elec_APDFT1(nuc_charges_vec)

13.195618303068606

In [11]:
ac_mol.get_relative_elec_APDFT3(nuc_charges_vec)

13.169787901189718

In [12]:
ac_mol.ap.build_elec_gradient()

array([-18.54581983, -25.14362898])

In [13]:
analytical_hessian = ac_mol.ap.build_elec_hessian()
print(analytical_hessian)

[[-2.90249887 -0.02931986]
 [-0.02931986 -3.54914235]]


In [14]:
if ac_mol.flag_response_property:
    print(ac_mol.get_elec_electric_dipole_APDFT0(nuc_charges_vec))

[2.46120379e-16 2.63754639e-15 8.30480604e-02]


In [15]:
if ac_mol.flag_response_property:
    print(ac_mol.get_elec_electric_dipole_APDFT1(nuc_charges_vec))

[2.29110318e-15 2.11232864e-15 1.88434219e+00]


In [16]:
if ac_mol.flag_response_property:
    print(ac_mol.get_electric_dipole_APDFT0(nuc_charges_vec))

[ 2.46120379e-16  2.63754639e-15 -1.99565068e+00]


In [17]:
if ac_mol.flag_response_property:
    print(ac_mol.get_electric_dipole_APDFT1(nuc_charges_vec))

[ 2.29110318e-15  2.11232864e-15 -1.94356552e-01]


In [18]:
if ac_mol.flag_response_property:
    print(ac_mol.get_elec_electric_polarizability_APDFT0(nuc_charges_vec))

[[ 1.05017980e+01 -7.32747196e-15  8.93849667e-15]
 [-7.32747196e-15  1.05017980e+01  1.89884688e-14]
 [ 8.93849667e-15  1.89884688e-14  1.42035431e+01]]


In [19]:
if ac_mol.flag_response_property:
    print(ac_mol.get_elec_electric_polarizability_APDFT1(nuc_charges_vec))

[[ 6.70126025e+00 -2.90057920e-15 -1.97613676e-14]
 [-2.90057920e-15  6.70126025e+00 -2.68895351e-14]
 [-1.97613676e-14 -2.68895351e-14  1.49342636e+01]]


In [20]:
if ac_mol.flag_response_property:
    analytical_dipole_gradient = ac_mol.ap.build_elec_electric_dipole_gradient()
    print(analytical_dipole_gradient)

[[ 9.22378053e-16 -2.58749940e-15 -5.97727535e-01]
 [ 2.96736086e-15 -3.11271715e-15  1.20356659e+00]]


Fractional charges calculations

In [21]:
# Without dm0=self.mf.init_guess_by_1e(), the error occurs

# ac_mol_fc = ac(target_mol, name_basis_set, dft_functional, sites=[0,1],
#                fractional_charges_calc=True, fractional_charges=[-0.001, 0.001])
ac_mol_fc = ac(target_mol, name_basis_set, dft_functional, sites=[0,1], guess="1e",
               fractional_charges_calc=True, fractional_charges=[-0.001, 0.001])

/home/takafumi/program/pyscf/pyscf/gto/mole.py:1213: UserWarning: Function mol.dumps drops attribute with_rinv_at_nucleus because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -113.237199661085


In [22]:
ac_mol_fc.ap.build_elec_gradient()

array([-18.54294679, -25.14714889])

In [23]:
ac_mol_fc.calc_all_derivatives()

In [24]:
import inspect
print(inspect.getfile(ac_mol_fc.mol.with_rinv_at_nucleus))

/home/takafumi/program/AQA/aqa/FcMole.py


Check the difference between analytical and numerical Hessian

In [25]:
ac_mol_fc1 = ac(target_mol, name_basis_set, dft_functional, sites=[0,1], guess="1e",
                fractional_charges_calc=True, fractional_charges=[0.001, 0.0], flag_response_property=True,
                charges_for_center=[6,8])
grad1 = ac_mol_fc1.ap.build_elec_gradient()
dip1 = ac_mol_fc1.ap.ref_elec_electric_dipole_moment * -1

ac_mol_fc2 = ac(target_mol, name_basis_set, dft_functional, sites=[0,1], guess="1e",
               fractional_charges_calc=True, fractional_charges=[-0.001, 0.0], flag_response_property=True,
               charges_for_center=[6,8])
grad2 = ac_mol_fc2.ap.build_elec_gradient()
dip2 = ac_mol_fc2.ap.ref_elec_electric_dipole_moment * -1

print("Finite difference gradient: ", (grad1 - grad2) / 0.002)
numerical_hessian = (grad1 - grad2) / 0.002

numerical_dipole_gradient = (dip1 - dip2) / 0.002

converged SCF energy = -113.244334745095
converged SCF energy = -113.214940227739
Finite difference gradient:  [-2.90249176 -0.02933379]


In [26]:
analytical_hessian[0] - numerical_hessian

array([-7.10974405e-06,  1.39281187e-05])

In [27]:
analytical_dipole_gradient[0] - numerical_dipole_gradient

array([ 1.64299456e-13, -2.03243240e-12, -2.57501389e-05])